In [191]:
!pip install numpy pytz

In [192]:
import pytz as tz
import yfinance as yf
import pandas as pd 
from datetime import datetime, timedelta

In [193]:
portfolio = {
    'CPE.N': {'ticker': 'CEIX', 'weight': 0.27},
    'HLX.N': {'ticker': 'HP', 'weight': 0.12},
    'DINO.N': {'ticker': 'LPG', 'weight': 0.22},
    'MTDR.N': {'ticker': 'MTDR', 'weight': 0.13},
    'MUR.N': {'ticker': 'MUR', 'weight': 0.14},
    'SWN.N': {'ticker': 'SM', 'weight': 0.12}
}

# Define benchmark (e.g., S&P 500)
benchmark_ticker = "^GSPC"  # S&P 500 Index

In [194]:
# Define the timezone
tz = tz.UTC
# Make time periods timezone-aware
one_month_ago = tz.localize(datetime(2024, 10, 1))
ytd_start = tz.localize(datetime(2024, 1, 1))
twelve_months_ago = tz.localize(datetime(2023, 11, 1))
today = tz.localize(datetime(2024, 11, 1))


In [195]:
# Download stock and benchmark data
tickers = [info['ticker'] for info in portfolio.values()]
all_tickers = tickers + [benchmark_ticker]

data = yf.download(all_tickers, start=twelve_months_ago.strftime('%Y-%m-%d'), end=today.strftime('%Y-%m-%d'))['Adj Close']

[*********************100%***********************]  7 of 7 completed


In [196]:
# Calculate returns
returns = pd.DataFrame(index=tickers)

# 1M returns
returns['1M'] = data.loc[one_month_ago:].pct_change().iloc[1:].sum()

# YTD returns
returns['YTD'] = data.loc[ytd_start:].pct_change().iloc[1:].sum()

# 12M returns
returns['12M'] = data.pct_change(periods=len(data) - 1).iloc[-1]

# Benchmark returns
benchmark_returns = pd.Series({
    '1M': data[benchmark_ticker].loc[one_month_ago:].pct_change().iloc[1:].sum(),
    'YTD': data[benchmark_ticker].loc[ytd_start:].pct_change().iloc[1:].sum(),
    '12M': data[benchmark_ticker].pct_change(periods=len(data) - 1).iloc[-1]
})

In [197]:
# Excess returns (stock returns - benchmark returns)
returns['1M_Excess'] = returns['1M'] - benchmark_returns['1M']
returns['YTD_Excess'] = returns['YTD'] - benchmark_returns['YTD']
returns['12M_Excess'] = returns['12M'] - benchmark_returns['12M']

In [198]:
# Assuming Risk-Free Rate is 0.0428 (1 yr treasury bond)
risk_free_rate = 0.0428

# Abnormal returns (using CAPM: Return - (Risk-Free Rate + Beta * Market Excess Return))
# Here, we assume a beta of 1 for simplicity. Adjust based on your analysis.
returns['1M_Abnormal'] = returns['1M'] - (risk_free_rate + 1 * benchmark_returns['1M'])
returns['YTD_Abnormal'] = returns['YTD'] - (risk_free_rate + 1 * benchmark_returns['YTD'])
returns['12M_Abnormal'] = returns['12M'] - (risk_free_rate + 1 * benchmark_returns['12M'])

In [199]:
# Create the weights Series
weights = pd.Series(
    {info['ticker']: info['weight'] for info in portfolio.values()}
)


# Align weights with the stock tickers in the returns DataFrame
weights = weights.reindex(returns.index)


# Multiply the returns by weights and sum for portfolio-level returns
portfolio_returns = (returns[['1M', 'YTD', '12M']].mul(weights, axis=0)).sum()


In [200]:
# Display results
print("Individual Stock Returns:")
print(returns)

print("\nPortfolio Returns:")
print(portfolio_returns)

Individual Stock Returns:
            1M       YTD       12M  1M_Excess  YTD_Excess  12M_Excess  \
CEIX  0.076528  0.159273  0.152982   0.076617   -0.032114   -0.193323   
HP    0.067684  0.010787 -0.104636   0.067773   -0.180600   -0.450941   
LPG  -0.214812 -0.277087 -0.103133  -0.214724   -0.468474   -0.449437   
MTDR  0.037378 -0.042998 -0.132684   0.037467   -0.234385   -0.478989   
MUR  -0.087374 -0.252585 -0.276711  -0.087285   -0.443972   -0.623015   
SM    0.025924  0.151362  0.079083   0.026013   -0.040025   -0.267222   

      1M_Abnormal  YTD_Abnormal  12M_Abnormal  
CEIX     0.033817     -0.074914     -0.236123  
HP       0.024973     -0.223400     -0.493741  
LPG     -0.257524     -0.511274     -0.492237  
MTDR    -0.005333     -0.277185     -0.521789  
MUR     -0.130085     -0.486772     -0.665815  
SM      -0.016787     -0.082825     -0.310022  

Portfolio Returns:
1M    -0.022736
YTD   -0.039449
12M   -0.040439
dtype: float64


In [201]:
# Save individual stock returns to Excel
returns.to_excel('individual_stock_returns.xlsx', sheet_name='Stock Returns')

# Save portfolio returns to Excel
portfolio_returns.to_frame(name='Portfolio Returns').to_excel('portfolio_returns.xlsx', sheet_name='Portfolio Returns')

print("Results saved to Excel files:")
print("1. Individual stock returns saved as 'individual_stock_returns.xlsx'")
print("2. Portfolio returns saved as 'portfolio_returns.xlsx'")

Results saved to Excel files:
1. Individual stock returns saved as 'individual_stock_returns.xlsx'
2. Portfolio returns saved as 'portfolio_returns.xlsx'


In [203]:
# Calculate Standard Deviation
# Define the stocks
tickers = ["MUR", "CEIX", "LPG", "SM", "MTDR", "HP"]

# Define the time periods
end_date = "2024-11-01"  # Fixed end date
six_months_ago = (datetime.strptime(end_date, "%Y-%m-%d") - timedelta(days=6 * 30)).strftime("%Y-%m-%d")  # Approx 6 months
twelve_months_ago = (datetime.strptime(end_date, "%Y-%m-%d") - timedelta(days=12 * 30)).strftime("%Y-%m-%d")  # Approx 12 months

# Download data for 6 months
data_6m = yf.download(tickers, start=six_months_ago, end=end_date)["Adj Close"]

# Download data for 12 months
data_12m = yf.download(tickers, start=twelve_months_ago, end=end_date)["Adj Close"]

# Calculate daily returns for 6 months
daily_returns_6m = data_6m.pct_change().dropna()

# Calculate daily returns for 12 months
daily_returns_12m = data_12m.pct_change().dropna()

# Calculate standard deviation for each stock for 6 months
std_deviation_6m = daily_returns_6m.std()

# Calculate standard deviation for each stock for 12 months
std_deviation_12m = daily_returns_12m.std()

# Rank the stocks by standard deviation
ranked_6m = std_deviation_6m.sort_values(ascending=False)
ranked_12m = std_deviation_12m.sort_values(ascending=False)

# Display the ranked results
print("\nRanked by Standard Deviation (6 Months):")
print(ranked_6m)

print("\nRanked by Standard Deviation (12 Months):")
print(ranked_12m)

# Save the ranked results to an Excel file
output_file = "stock_volatility_ranking.xlsx"

# Create a dictionary to store the ranked results
results = {
    "6M Rank": ranked_6m,
    "12M Rank": ranked_12m
}

# Convert the dictionary to a DataFrame
ranked_df = pd.DataFrame(results)

# Save the DataFrame to Excel
ranked_df.to_excel(output_file, sheet_name="Volatility Ranking")

print(f"Ranked results saved to {output_file}")



[*********************100%***********************]  6 of 6 completed
[*********************100%***********************]  6 of 6 completed


Ranked by Standard Deviation (6 Months):
Ticker
SM      0.026920
HP      0.024129
CEIX    0.024114
LPG     0.023897
MTDR    0.022006
MUR     0.017942
dtype: float64

Ranked by Standard Deviation (12 Months):
Ticker
LPG     0.025607
CEIX    0.025220
SM      0.023369
HP      0.022767
MTDR    0.020332
MUR     0.017226
dtype: float64
Ranked results saved to stock_volatility_ranking.xlsx
